In [1]:
# Librerias

import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import os
import requests
from time import sleep, strftime
import random
from random import randint
from bs4 import BeautifulSoup
from selenium import webdriver
import shutil
import pickle

In [2]:
# Funciones de transformacion

def libras_kg(peso):
    
    try:
        if peso.split()[1] == 'lb':
            peso = float(peso.split()[0])
            kg = peso/2.2046

        elif peso.split()[1] == 'kg':
            kg = peso.split()[0]
        
    except:
        kg = np.nan

    return kg

def altura_cm(altura):
    
    try:
        if float(altura) < 100:
            pies = int(altura.split('.')[0])
            pies_cm = pies/0.032808
            pulgadas = int(altura.split('.')[1])
            pulgadas_cm = pulgadas * 2.54
            cm = pies_cm + pulgadas_cm

        elif float(altura) > 100:
            cm = altura.split()[0]
            cm = float(altura)

    except:
        cm = np.nan

    return cm 


In [ ]:
#Abro el navegador
navegador = webdriver.Chrome(executable_path='/home/nestor/Proyectos/neoland/Proyecto Final/chromedriver')

# Buscar pagina

sleep(randint(4,7))
navegador.get("https://games.crossfit.com/leaderboard/open/2020?view=0&division=1&scaled=0&sort=0&page=31") 


# Superar politica de privacidad y subscripcion
sleep(randint(2,5))
navegador.find_element_by_partial_link_text('ACCEPT').click()
sleep(randint(2,5))
navegador.find_element_by_class_name('mc-closeModal').click()

# Creo lista vacía para luego convertirla en dataframe

filas = list()

# Bucle para ir pasando de páginas 

for j in range(31,40):
    
    try:
        if j > 20:
            navegador.find_element_by_xpath('//*[@id="leaderboardSponsorVisible"]/div/div[3]/div/div[2]/a[5]').click()
            sleep(randint(4,8))
        else:
            pass
    except:
        pass
    
    # Bucle para ir recorriendo cada uno de los deportistas
    for i in range(1,51): 
        
        try:
        
            # Recojo la info de la tabla de la pag principal
            sleep(randint(4,8))
            puntos = float(navegador.find_element_by_xpath('//*[@id="leaderboardSponsorVisible"]/div/table/tbody/tr['+str(i)+']/td[3]/div').text)
            wod_1 = navegador.find_element_by_xpath('//*[@id="leaderboardSponsorVisible"]/div/table/tbody/tr['+str(i)+']/td[4]/div').text.split()[1].replace('(', '').replace(')', '')
            wod_2 = navegador.find_element_by_xpath('//*[@id="leaderboardSponsorVisible"]/div/table/tbody/tr['+str(i)+']/td[5]/div').text.split()[1].replace('(', '').replace(')', '')
            wod_3 = navegador.find_element_by_xpath('//*[@id="leaderboardSponsorVisible"]/div/table/tbody/tr['+str(i)+']/td[6]/div').text.split()[1].replace('(', '').replace(')', '')
            wod_4 = navegador.find_element_by_xpath('//*[@id="leaderboardSponsorVisible"]/div/table/tbody/tr['+str(i)+']/td[7]/div').text.split()[1].replace('(', '').replace(')', '')
            wod_5 = navegador.find_element_by_xpath('//*[@id="leaderboardSponsorVisible"]/div/table/tbody/tr['+str(i)+']/td[8]/div').text.split()[1].replace('(', '').replace(')', '')




            # Voy abriendo la página del crosfitero

            sleep(randint(5,8))
            navegador.find_element_by_xpath('//*[@id="leaderboardSponsorVisible"]/div/table/tbody/tr['+str(i)+']/td[2]/div/div[1]/div[1]').click()
            sleep(randint(7,9))
            navegador.find_element_by_xpath('//*[@id="leaderboardSponsorVisible"]/div/table/tbody/tr['+str(i)+']/td[2]/div/div[2]/ul/a').click()


            ## Preparo la sopa

            sleep(randint(5,9))
            sopa = BeautifulSoup(navegador.page_source)



            ## Cojo la info del edportista

            #########################################################
            # Nombre
            nombre = sopa.find('h3').text.split('\n')[1]
            # Apellido
            apellido = sopa.find('h3').text.split('\n')[2].strip()
            # Edad
            edad = float(sopa.find('div', attrs = {'class':'text lg'}).text.replace('\n','').strip())
            # Altura (modificar a cm)
            altura = [i.text for i in sopa.find_all('div', attrs = {'class': 'text lg nowrap'})][0].strip().replace('\'', ',').replace('"','').replace(',','.')
            altura = altura_cm(altura.split()[0])
            # Peso
            peso = [i.text for i in sopa.find_all('div', attrs = {'class': 'text lg nowrap'})][1].strip()
            peso = libras_kg(peso)


            # Back squat
            back_squat = sopa.find('div', attrs = {'class':'stats-section'}).text.split('\n')[5].strip()
            back_squat = libras_kg(back_squat)
            # Clean and jerck
            clean_jerk = sopa.find('div', attrs = {'class':'stats-section'}).text.split('\n')[9].strip()
            clean_jerk = libras_kg(clean_jerk)
            # Snatch
            snatch = sopa.find('div', attrs = {'class':'stats-section'}).text.split('\n')[13].strip()
            snatch = libras_kg(snatch)


            # Deadlift
            deadlift = [i for i in sopa.find_all('div', attrs = {'class':'stats-section'})][1].text.split('\n')[5].strip()
            deadlift = libras_kg(deadlift)
            # Fight gone bad
            fight_gone_bad = [i for i in sopa.find_all('div', attrs = {'class':'stats-section'})][1].text.split('\n')[9].strip()
            # Max Pull-ups
            pull_ups = [i for i in sopa.find_all('div', attrs = {'class':'stats-section'})][1].text.split('\n')[13].strip()


            # Fran
            fran = [i for i in sopa.find_all('div', attrs = {'class':'stats-section'})][2].text.split('\n')[5].strip()
            # Grace
            grace = [i for i in sopa.find_all('div', attrs = {'class':'stats-section'})][2].text.split('\n')[9].strip()
            # Helen
            helen = [i for i in sopa.find_all('div', attrs = {'class':'stats-section'})][2].text.split('\n')[13].strip()
            # Filthy 50
            filthy = [i for i in sopa.find_all('div', attrs = {'class':'stats-section'})][3].text.split('\n')[5].strip()
            # Sprint 400m
            sprint = [i for i in sopa.find_all('div', attrs = {'class':'stats-section'})][3].text.split('\n')[9].strip()
            # Run 5k
            run_5k = [i for i in sopa.find_all('div', attrs = {'class':'stats-section'})][3].text.split('\n')[13].strip()

            #########################################################

            # Guardo la info en una lista

            filas.append([nombre, apellido, edad, altura, peso,
                   back_squat, clean_jerk, snatch,
                   deadlift, fight_gone_bad, pull_ups,
                   fran, grace, helen, filthy, sprint, run_5k, puntos, wod_1, wod_2, wod_3,
                    wod_4, wod_5])

            sleep(randint(2,5))
            navegador.back()
            sleep(randint(3,7))
        
        except:
            pass

In [ ]:
# Creo el data frame

columnas = ['nombre', 'apellido', 'edad', 'altura', 'peso',
           'back_squat', 'clean_jerk', 'snatch',
           'deadlift', 'fight_gone_bad', 'pull_ups',
           'fran', 'grace', 'helen', 'filthy', 'sprint', 'run_5k',
           'puntos', 'wod_1', 'wod_2', 'wod_3',
           'wod_4', 'wod_5']

prueba2 = pd.DataFrame(filas, 
                      columns = columnas)

# Guardo el data frame en un csv
prueba2.to_csv('muestra3_51-60')